# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770401262767                   -0.52    9.0    178ms
  2   -2.771691139498       -2.89       -1.33    1.0   93.8ms
  3   -2.771714194484       -4.64       -2.41    1.0    508ms
  4   -2.771714629262       -6.36       -3.10    1.0   92.9ms
  5   -2.771714715191       -7.07       -4.62    2.0    136ms
  6   -2.771714715235      -10.36       -4.80    1.0    129ms
  7   -2.771714715247      -10.91       -5.53    1.0   97.1ms
  8   -2.771714715250      -11.57       -6.33    2.0    129ms
  9   -2.771714715250      -13.44       -7.70    1.0   99.4ms
 10   -2.771714715250   +  -14.21       -8.07    2.0    113ms


-0.00013457326596966047

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770492856027                   -0.52    9.0    224ms
  2   -2.771778976241       -2.89       -1.32    1.0    145ms
  3   -2.771801511129       -4.65       -2.42    1.0   95.9ms
  4   -2.771802002175       -6.31       -3.13    1.0    100ms
  5   -2.771802073972       -7.14       -4.33    2.0    557ms
  6   -2.771802074266       -9.53       -4.55    1.0    117ms
  7   -2.771802074469       -9.69       -5.26    1.0    132ms
  8   -2.771802074474      -11.38       -5.40    1.0    129ms
  9   -2.771802074476      -11.61       -6.25    1.0    102ms
 10   -2.771802074476      -13.25       -6.24    2.0    119ms
 11   -2.771802074476      -13.39       -6.62    1.0    114ms
 12   -2.771802074476      -15.05       -7.34    1.0    105ms
 13   -2.771802074476   +  -13.94       -8.17    1.0    118ms


0.01761222142458377

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457326596966047
Displaced dipole:  0.01761222142458377
Polarizability :   1.774679469055343


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920632782e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766550999307e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762144474e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593778441e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787450916e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.274013190253e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.754557013854e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.754557013854e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.535621101852e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.826157562437e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.854910196576e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.000409871685e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.000409871685e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.